In [1]:
# Let's define the structure of the decision tree as a Python class and calculate the EMV.

class InvestmentNode:
    def __init__(self, action, stock_up_prob=None, outcomes=None, children=None):
        """
        Initialize the decision tree node.
        :param action: The action taken ('stock' or 'bank').
        :param stock_up_prob: The probability of stock going up for the node.
        :param outcomes: A dictionary of outcomes with their respective monetary values.
        :param children: A dictionary of children nodes keyed by stock movement ('up' or 'down').
        """
        self.action = action
        self.stock_up_prob = stock_up_prob
        self.outcomes = outcomes or {}
        self.children = children or {}
        self.emv = 0  # Expected Monetary Value

    def calculate_emv(self):
        """
        Calculate the EMV recursively for each node.
        """
        # If there are no children, the EMV is simply the outcome of the current action.
        if not self.children:
            self.emv = self.outcomes.get('any', 0)
            return self.emv
        
        # If there are children nodes, calculate the EMV considering the probabilities and outcomes.
        up_emv = self.children['up'].calculate_emv()
        down_emv = self.children['down'].calculate_emv()
        
        # EMV calculation based on the stock going up or down.
        self.emv = (self.stock_up_prob * up_emv) + ((1 - self.stock_up_prob) * down_emv)
        return self.emv

    def __str__(self):
        """
        String representation of the node for debugging purposes.
        """
        return f"Action: {self.action}, EMV: {self.emv}"

# Now let's build the tree with the given probabilities and outcomes.
# First period decision node
first_period_stock = InvestmentNode(
    action='stock',
    stock_up_prob=0.4,  # Probability of stock going up in the first period
    children={
        'up': InvestmentNode(  # If stock goes up in the first period
            action='stock',
            stock_up_prob=0.35,  # P(2nd = up|1st = up)
            children={
                'up': InvestmentNode(action='stock', outcomes={'any': 22}),  # Outcome if stock goes up again
                'down': InvestmentNode(action='stock', outcomes={'any': 12.5})  # Outcome if stock goes down
            }
        ),
        'down': InvestmentNode(  # If stock goes down in the first period
            action='stock',
            stock_up_prob=0.45,  # P(2nd = up|1st = down)
            children={
                'up': InvestmentNode(action='stock', outcomes={'any': 11}),  # Outcome if stock goes up
                'down': InvestmentNode(action='stock', outcomes={'any': 6})  # Outcome if stock goes down
            }
        )
    }
)

# Second period decision node for the bank
second_period_bank = InvestmentNode(
    action='bank',
    outcomes={'any': 10.5}
)

# Second period decision node for stock, coming from a bank investment in the first period
second_period_stock_from_bank = InvestmentNode(
    action='stock',
    stock_up_prob=0.4,  # Assuming the probability is the same as if it was the first investment
    children={
        'up': InvestmentNode(action='stock', outcomes={'any': 13.5}),  # Outcome if stock goes up
        'down': InvestmentNode(action='stock', outcomes={'any': 8})  # Outcome if stock goes down
    }
)

# First period decision node for bank
first_period_bank = InvestmentNode(
    action='bank',
    children={
        'up': second_period_stock_from_bank,  # If deciding to go for stock in the second period
        'down': second_period_bank  # If deciding to go for bank again in the second period
    }
)

# Calculate the EMV for each decision node
first_period_stock.calculate_emv()
first_period_bank.calculate_emv()

# Now, let's find out which option has the higher EMV.
optimal_decision = 'stock' if first_period_stock.emv > first_period_bank.emv else 'bank'
optimal_emv = max(first_period_stock.emv, first_period_bank.emv)

(optimal_decision, optimal_emv, first_period_stock.emv, first_period_bank.emv)


TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

In [2]:
# Let's write a Python code that calculates the expected monetary value (EMV) and the expected utility for the investment decision problem.

import math

# Define the monetary outcomes
outcomes = {
    'stock_up_stock_up': 22,
    'stock_up_stock_down': 12.5,
    'stock_up_bank': 14.5,
    'stock_down_stock_up': 11,
    'stock_down_stock_down': 6,
    'stock_down_bank': 8,
    'bank_stock_up': 13.5,
    'bank_stock_down': 8,
    'bank_bank': 10.5
}

# Define the probabilities
prob_stock_up_first = 0.4
prob_stock_up_given_up = 0.35
prob_stock_up_given_down = 0.45

# Utility function
def utility(x):
    return math.sqrt(x / 1000)

# Calculate expected monetary value and utility for the second investment period
def calculate_second_period(stock_up):
    # If the first action is investing in stock and it went up
    if stock_up:
        emv_stock = prob_stock_up_given_up * outcomes['stock_up_stock_up'] + \
                    (1 - prob_stock_up_given_up) * outcomes['stock_up_stock_down']
        emv_bank = outcomes['stock_up_bank']
    # If the first action is investing in stock and it went down
    else:
        emv_stock = prob_stock_up_given_down * outcomes['stock_down_stock_up'] + \
                    (1 - prob_stock_up_given_down) * outcomes['stock_down_stock_down']
        emv_bank = outcomes['stock_down_bank']

    # Calculate utility
    utility_stock = utility(emv_stock)
    utility_bank = utility(emv_bank)

    return emv_stock, emv_bank, utility_stock, utility_bank

# Calculate expected monetary value and utility for the first investment period
def calculate_first_period():
    # Calculate for stock
    emv_stock_up, _, utility_stock_up, _ = calculate_second_period(stock_up=True)
    emv_stock_down, _, utility_stock_down, _ = calculate_second_period(stock_up=False)
    
    # EMV for choosing stock in the first period
    emv_stock_first = prob_stock_up_first * emv_stock_up + (1 - prob_stock_up_first) * emv_stock_down
    # Utility for choosing stock in the first period
    utility_stock_first = prob_stock_up_first * utility_stock_up + (1 - prob_stock_up_first) * utility_stock_down

    # Calculate for bank
    _, emv_bank_up, _, utility_bank_up = calculate_second_period(stock_up=True)  # same as bank_down
    # EMV for choosing bank in the first period
    emv_bank_first = outcomes['bank_bank']
    # Utility for choosing bank in the first period
    utility_bank_first = utility(emv_bank_first)

    return emv_stock_first, emv_bank_first, utility_stock_first, utility_bank_first

# Calculate EMV and utility for the entire decision tree
def calculate_overall():
    emv_stock, emv_bank, utility_stock, utility_bank = calculate_first_period()
    
    # Choose the option with the higher EMV
    optimal_emv_action = 'stock' if emv_stock > emv_bank else 'bank'
    optimal_emv = max(emv_stock, emv_bank)

    # Choose the option with the higher utility
    optimal_utility_action = 'stock' if utility_stock > utility_bank else 'bank'
    optimal_utility = max(utility_stock, utility_bank)

    return {
        'optimal_emv_action': optimal_emv_action,
        'optimal_emv': optimal_emv,
        'optimal_utility_action': optimal_utility_action,
        'optimal_utility': optimal_utility,
        'emv_stock': emv_stock,
        'emv_bank': emv_bank,
        'utility_stock': utility_stock,
        'utility_bank': utility_bank
    }

# Execute the calculation
results = calculate_overall()
results


{'optimal_emv_action': 'stock',
 'optimal_emv': 11.280000000000001,
 'optimal_utility_action': 'stock',
 'optimal_utility': 0.10481668887539136,
 'emv_stock': 11.280000000000001,
 'emv_bank': 10.5,
 'utility_stock': 0.10481668887539136,
 'utility_bank': 0.10246950765959599}